In [1]:
import json
import openai
from bs4 import BeautifulSoup as bs
import config

In [2]:
# Reading the API key from a file
service_key = config.api_key
openai.api_key = service_key

In [3]:
# TODO: Parse the correct URL
with open("myresponse.html") as fp:
    soup = bs(fp, "html.parser")

In [4]:
# Parse the HTML for the data
data_set = []
for tag in soup.find_all('div', class_='number'):
    data_set.append(tag.text)
for tag in soup.find_all('ul', class_='scores'):
    for ele in tag.find_all('li'):
        data_set.append(ele.text)
for tag in soup.find_all('dl', class_='stats'):
    for ele in tag.find_all('div'):
        text = ele.text.replace('\n', '')
        text = " ".join(text.split())
        data_set.append(text)

In [5]:
data_set

['Package Health Score 95 / 100',
 'Popularity Key ecosystem project',
 'Maintenance Healthy',
 'Security No known security issues',
 'Community Active',
 'Dependents 156.9K',
 'GitHub Stars 169.02K',
 'Forks 34.08K',
 'Contributors 430',
 'Open Issues 531',
 'Open PR 188',
 'Last Release 2 months ago',
 'Last Commit 2 days ago',
 'Readme.md Yes',
 'Contributing.md Yes',
 'Code of Conduct Yes',
 'Contributors 430',
 'Funding No',
 'Node.js Compatibility >=0.10.0',
 'Age 10 years',
 'Dependencies 2 Direct',
 'Versions 453',
 'Install Size 291 kB',
 'Dist-tags 3',
 '# of Files 18',
 'Maintainers 8',
 'TS Typings Yes']

In [6]:
# Create list of JSON objects
list_of_jsons = []
for line in data_set:
    json_obj = {"text": line}
    list_of_jsons.append(json_obj)

In [7]:
# Create file with data
with open('data.jsonl', 'w') as json_file:
    for ele in list_of_jsons:
        json.dump(ele, json_file)
        json_file.write('\n')

In [8]:
# Send file to openAI
openai.File.create(file=open("data.jsonl"), purpose='answers')

Upload progress: 100%|██████████| 1.08k/1.08k [00:00<00:00, 2.33kit/s]


<File file id=file-YucxNFvPYSOAlsaLzE77j9iH at 0x7f023faf6ae0> JSON: {
  "bytes": 840,
  "created_at": 1622322976,
  "filename": "data.jsonl",
  "id": "file-YucxNFvPYSOAlsaLzE77j9iH",
  "object": "file",
  "purpose": "answers",
  "status": "uploaded",
  "status_details": null
}

In [14]:
# Get an answer
openai.Answer.create(
    search_model="ada", 
    model="curie", 
    question="Can I build website with this package?", 
    file="file-YucxNFvPYSOAlsaLzE77j9iH", 
    examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
    examples=[["What is human life expectancy in the United States?", "78 years."]], 
    max_rerank=10,
    max_tokens=5,
    stop=["\n", "<|endoftext|>"]
)

<OpenAIObject answer at 0x7f023f08fb30> JSON: {
  "answers": [
    "Yes, you can."
  ],
  "completion": "cmpl-350dIT4440KbFHBTw1xQbkZXtEDru",
  "file": "file-YucxNFvPYSOAlsaLzE77j9iH",
  "model": "curie:2020-05-03",
  "object": "answer",
  "search_model": "ada:2020-05-03",
  "selected_documents": [
    {
      "document": 0,
      "object": "search_result",
      "score": 56.659,
      "text": "Package Health Score 95 / 100"
    }
  ]
}